In [1]:
# Ументшить размер каждого вектора, 300 слишком большой вектор

### TECHNOLOGIES:

DBScan + Word2Vec

### IMPROT MODULES:

In [2]:
import os
import re
import csv
import nltk
import string
import random
import tweepy
import gettext
import pycountry

import numpy as np
import pandas as pd
import gensim.downloader as api

from nltk import word_tokenize
from tweepy import OAuthHandler
from nltk.corpus import stopwords
from gensim.models import Word2Vec
from sklearn.cluster import DBSCAN
from sklearn.cluster import MiniBatchKMeans
from sklearn.metrics import silhouette_samples, silhouette_score

SEED = 42

random.seed(SEED)
np.random.seed(SEED)

nltk.download("punkt")
nltk.download("stopwords")
os.environ["PYTHONHASHSEED"] = str(SEED)

custom_stopwords = set(stopwords.words('russian'))

[nltk_data] Downloading package punkt to /home/deniz/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/deniz/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### DATA:

In [3]:
# Add more
replace_items = {
                # Предлоги
                ' в ': ' ', ' к ': ' ', " до ": ' ', " по ": ' ', " через ": ' ', " после ": ' ', " в течение ": ' ', 
                 " в продолжение ": ' ', " в заключение ": ' ', " из-за ": ' ', " над ": ' ', " под ": ' ', " перед ": ' ', 
                ' у ': ' ', " возле ": ' ', " мимо ": ' ', " около ": ' ', " от ": ' ', " ради ": ' ', " благодаря ": ' ', 
                " в силу ": ' ', " ввиду ": ' ', " вследствие ": ' ', " для ": ' ',  " на ": ' ', " в целях ": ' ', 
                " с целью ": ' ', " вопреки ": ' ', " несмотря на ": ' ', ' о ': ' ', " об ": ' ', " обо ": ' ', 
                " про ": ' ', " насчёт ": ' ', ' с ': ' ', " вроде ": ' ', " наподобие ": ' ', " как ": ' ', " без ": ' ',
                # Союзы
                 " что ": ' ', " когда ": ' ', " ибо ": ' ', " пока ": ' ', " будто ": ' ', " словно ": ' ', " если ": ' ', " кто ": ' ', 
                 " который ": ' ', " какой ": ' ', " где ": ' ', " куда ": ' ', " откуда ": ' ', " потому что ": ' ', " оттого что ": ' ', 
                 " так как ": ' ', " так что ": ' ', " лишь только ": ' ', " как будто ": ' ', " с тех пор как ": ' ', 
                 " в связи с тем что ": ' ', " для того чтобы ": ' ', ' и ': ' ', " да ": ' ', " ни ни ": ' ', " тоже ": ' ', 
                 " также ": ' ', " не только ": ' ', " не столько ": ' ', " не то чтобы ": ' ', " или ": ' ', " лишь ": ' ', 
                 " либо ": ' ', " то ": ' ', " не то ": ' ', " а ": ' ', " но ": ' ', " зато ": ' ', " однако ": ' ', " то есть ": ' ', 
                 " а именно ": ' ', " чтоб ": ' ', " чтобы ": ' ', " с тех пор ": ' ', " едва ": ' ', " прежде чем ": ' ', 
                 " перед тем как ": ' ', " точно ": ' ', " подобно ": ' ', " дабы ": ' ', " коли ": ' ', " ежели ": ' ', " раз ": ' ', 
                 " пускай ": ' ', " хотя ": ' ', " поэтому ": ' ',
                 # Другие слова
                 " шт ": ' ', " мл ": ' ', " мг ": ' ', " премиум ": ' ', " вес ": ' ', " из ": ' ', " спб ": ' ',
                 " вкусвилл ": ' ', " стандарту ": ' ', " вв ": ' ', " ве ": ' ', ' г ': ' ', ' гр ': ' ', " молодой ": ' ',
                 " сладкий ": ' ', " мини ": ' ', " кг ": ' ',
                 # Страны 
                 " абхазия ": ' ', " албания ": ' ', " армения ": ' ', " австралия ": ' ', " алжир ": ' ', " андорра ": ' ', 
                 " аруба ": ' ', " австрия ": ' ', " афганистан ": ' ', " азербайджан ": ' ', " аргентина ": ' ', 
                 " багамы ": ' ', " белиз ": ' ', " босния и герцеговина ": ' ', " бруней ": ' ',
                 " бангладеш ": ' ', " белоруссия ": ' ', " болгария ": ' ', " вьетнам ": ' ', " венгрия ": ' ', 
                 " габон ": ' ', " гана ": ' ', " греция ": ' ', " германия ": ' ', " грузия ": ' ', " гондурас ": ' ', 
                 " гватемала ": ' ', " гаити ": ' ', " гайана ": ' ', " ботсвана ": ' ', " барбадос ": ' ', 
                 " бельгия ": ' ', " боливия ": ' ', " бразилия ": ' ', " бурунди ": ' ', " бутан ": ' ', 
                 " бахрейн ": ' ', " гонконг ": ' ', " бенин ": ' ', " бонэйр ": ' ', " вануату ": ' ', 
                 " венесуэла ": ' ', " ватикан ": ' ', " великобритания ": ' ', " египет ": ' ', " замбия ": ' ', 
                 " зимбабве ": ' ', " израиль ": ' ', " ирак ": ' ', " испания ": ' ', " индия ": ' ', " италия ": ' ', 
                 " иран ": ' ', " индонезия ": ' ', " ирландия ": ' ', " иордания ": ' ', " исландия ": ' ',
                 " йемен ": ' ', " канада ": ' ', " киргизия ": ' ', " корея ": ' ', " кувейт ": ' ', 
                 " казахстан ": ' ', " катар ": ' ', " кирибати ": ' ', " коста-рика ": ' ', " кюрасао ": ' ', 
                 " китай ": ' ', " кения ": ' ', " камбоджа ": ' ', " конго ": ' ', " камерун ": ' ', " кипр ": ' ', 
                 " колумбия ": ' ', " кндр ": ' ', " куба ": ' ', " лаос ": ' ', " ливан ": ' ', " люксембург ": ' ',
                 " гвинея ": ' ', " гренада ": ' ', " гамбия ": ' ', " дания ": ' ', " доминика ": ' ', " конго ": ' ', 
                 " доминикана ": ' ', " латвия ": ' ', " ливия ": ' ', " лесото ": ' ', " литва ": ' ', " либерия ": ' ',
                 " лихтенштейн ": ' ', " маврикий ": ' ', " малави ": ' ', " мальдивы ": ' ', " мозамбик ": ' ', 
                 " мьянма ": ' ', " мавритания ": ' ', " марокко ": ' ', " малайзия ": ' ', " марокко ": ' ', 
                 " молдавия ": ' ', " мадагаскар ": ' ', " мали ": ' ', " монако ": ' ', " македония ": ' ', 
                 " мальта ": ' ', " мексика ": ' ', " монголия ": ' ', " намибия ": ' ', " нигерия ": ' ', 
                 " норвегия ": ' ', " науру ": ' ', " нидерланды ": ' ', " непал ": ' ', " никарагуа ": ' ', 
                 " нигер ": ' ', " новая зеландия ": ' ', " оаэ ": ' ', " оман ": ' ', " пакистан ": ' ', " палау ": ' ', 
                 " панама ": ' ', " польша ": ' ', " перу ": ' ', " парагвай ": ' ', " португалия ": ' ', " россия ": ' ', 
                 " руанда ": ' ', " румыния ": ' ', " саба ": ' ', " сенегал ": ' ', " сирия ": ' ', " сальвадор ": ' ', 
                 " саудовская Аравия ": ' ', " сербия ": ' ', " словакия ": ' ', " сомали ": ' ', " самоа ": ' ', 
                 " свазиленд ": ' ', " сингапур ": ' ', " словения ": ' ', " судан ": ' ', " сейшелы ": ' ', 
                 " сша ": ' ', " суринам ": ' ', " таджикистан ": ' ', " токелау ": ' ', " тунис ": ' ', " тонга ": ' ',
                 " таиланд ": ' ', " турция ": ' ', " танзания ": ' ', " того ": ' ', " тувалу ": ' ', " узбекистан ": ' ',
                 " украина ": ' ', " финляндия ": ' ', " фиджи ": ' ', " франция ": ' ', " филиппины ": ' ',
                 " туркменистан ": ' ', " уганда ": ' ', " уругвай ": ' ', " швейцария ": ' ', " швеция ": ' ',
                 " эритрея ": ' ', " эстония ": ' ', " юар ": ' ', " япония ": ' ', " ямайка ": ' ', 
                 " хорватия ": ' ', " черногория ": ' ', " чехия ": ' ', " чили ": ' ', " шри-ланка ": ' ', 
                 " эквадор ": ' ', " эфиопия ": ' ',
                 }

### TAKE DATA:

In [4]:
def check_categorie(df, categorie):
    result = pd.DataFrame();
    
    for line in range(len(df.index)):
        if str(categorie) in df["categories_array"][line]:
            df_string = pd.DataFrame({"item_id": [df["item_id"][line]], "item_name": [df["item_name"][line]]})
            result = pd.concat([result, df_string], ignore_index=True)

    return result

In [5]:
with open("../data/vkusvill_items.csv", 'r', encoding='utf-8') as file:
    reader = csv.DictReader(file)
    catalog_df = pd.DataFrame(reader)

catalog_df = catalog_df.rename(columns = {"": "shop_id", "item_composition_txt": "item_composition", "nutrion_value_txt" : "nutrion_value"})
catalog = catalog_df.drop(["shop_id", "price", "vat", "measure_unit", "measure_value", "protein_value", "fat_value", "carb_value", 
                           "energy_value", "measure_quantum"], axis=1)

catalog_df = check_categorie(catalog_df, 10068)  # Овощи
catalog_df

,item_id,item_name
0,41728,Картофель молодой Египет
1,43208,"Томаты черри сладкая гроздь, 250 г"
2,44309,Капуста белокочанная резаная \Свежая капуста\
3,44310,Морковь резаная \Сочная морковь\
4,44311,Свекла резаная \Сладкая свекла\
...,...,...
144,31268,"Имбирь, 200 г"
145,21494,Перец оранжевый сладкий
146,75879,"Горох стручковый сладкий, 250 г"
147,60005,"Перец оранжевый сладкий, 450 г"


### CLEAN AND PREPARE DATA:

In [6]:
def delete_duplicate(lst):
    answer = set(lst.split())
    answer = list(answer)
    return " ".join(answer)

In [7]:
def delete_adjectives(str):
    lst = str.split()
    answer_lst = []
    
    for i in lst:
        if not i[-2:] in ["ая", "яя", "ое", "ее", "ие", "ые", "им", "ым", "ую", "юю", "ой", "ей", "ый", "ий"]:
            answer_lst.append(i)

    answer_lst = " ".join(answer_lst)
    return answer_lst

In [8]:
def replace_all(text, replace_items):
    for i, j in replace_items.items():
        text = text.lower().replace(i, j)
        
    return text

In [9]:
def clear_item_name(item_name, replace_items=replace_items):
    answer = ""
    
    for symbol in item_name:
        if symbol == '_':
            answer += ' '  
        elif symbol in [
            '\\', 
            '(', 
            ')', 
            '.', 
            ',', 
            '%'
        ] or symbol.isdigit():
            answer += ' '
        else:
            answer += symbol

    answer = answer.lower() + ' '
    
    while answer != replace_all(answer, replace_items):
        answer = replace_all(answer, replace_items)
    
    while "  " in answer:
        answer = answer.replace("  ", ' ')

    answer = delete_duplicate(answer)
    answer = delete_adjectives(answer)
    return answer.lower()

In [10]:
# Добавляем грубо теги, дабы важные для нас слова находились (грубо, стоит проводить динамический тегизатор. 
# Пример: икра бывает как кабачка, так и рыбная, категории у них разные, так для прилагательных можно сделать 
# токенезатор по окончанию)


def add_noun(token):
    return token + "_NOUN"

new_catalog = catalog_df.copy()
new_catalog['item_name'] = new_catalog['item_name'].apply(clear_item_name)

### Clean and prepare data:

In [11]:
def clean_text(text, tokenizer, stopwords):
    global replace_items
    
    text = str(text).lower()  # Lowercase words
    text = re.sub(r"\[(.*?)\]", "", text)  # Remove [+XYZ chars] in content
    text = re.sub(r"\s+", " ", text)  # Remove multiple spaces in content
    text = re.sub(r"\w+…|…", "", text)  # Remove ellipsis (and last word)
    text = re.sub(r"(?<=\w)-(?=\w)", " ", text)  # Replace dash between words
    text = re.sub(
        f"[{re.escape(string.punctuation)}]", "", text
    )  # Remove punctuation

    # text = clear_item_name(text, replace_items)
    tokens = tokenizer(text)  # Get tokens from text
    tokens = [t for t in tokens if not t in custom_stopwords]  # Remove stopwords
    tokens = ["" if t.isdigit() else t for t in tokens]  # Remove digits
    tokens = [t for t in tokens if len(t) > 1]  # Remove short tokens
    tokens = [add_noun(t) for t in tokens if len(t) > 1]
    # print(tokens)
    return tokens

### TOKENIZE:

In [12]:
text_columns = ["item_name"]

for col in text_columns:
    new_catalog[col] = new_catalog[col].astype(str)

# Create text column based on item_name
new_catalog["text"] = new_catalog[text_columns].apply(lambda x: " | ".join(x), axis=1)
new_catalog["tokens"] = new_catalog["text"].map(lambda x: clean_text(x, word_tokenize, custom_stopwords))

# Remove duplicated after preprocessing
# _, idx = np.unique(new_catalog["tokens"], return_index=True)
# new_catalog = new_catalog.iloc[idx, :]

# Remove empty values and keep relevant columns
new_catalog = new_catalog.loc[new_catalog.tokens.map(lambda x: len(x) > 0), ["text", "tokens"]]
docs = new_catalog["text"].values
tokenized_docs = new_catalog["tokens"].values

### LEARN MODEL:

In [13]:
# NOUN, VERB, ADJ

In [14]:
# For studying model
# model = Word2Vec(sentences=tokenized_docs, vector_size=10, workers=1, seed=SEED)

In [15]:
wv = api.load('word2vec-ruscorpora-300')

In [16]:
# wv.most_similar("томат_NOUN")

In [17]:
def vectorize(list_of_docs, model):
    """Generate vectors for list of documents using a Word Embedding

    Args:
        list_of_docs: List of documents
        model: Gensim's Word Embedding

    Returns:
        List of document vectors
    """
    features = []

    for tokens in list_of_docs:
        zero_vector = np.zeros(model.vector_size)
        vectors = []
        
        for token in tokens:
            if token in wv:
                try:
                    vectors.append(wv[token])
                except KeyError:
                    continue
        if vectors:
            vectors = np.asarray(vectors)
            avg_vec = vectors.mean(axis=0)
            features.append(avg_vec)
        else:
            features.append(zero_vector)
            
    return features
    
vectorized_docs = vectorize(tokenized_docs, model=wv)
len(vectorized_docs), len(vectorized_docs[0])
dbscan_model = DBSCAN(eps = 0.15, min_samples = 1).fit(vectorized_docs)
result_labels = dbscan_model.labels_
labels = result_labels
labels.shape

(149,)

### USE DBScan:

In [18]:
clusters3 = labels

out = { 'title': catalog_df["item_name"], 'cluster': clusters3 }

clusters3 = pd.Series(clusters3)
new_catalog = pd.concat([catalog_df, new_catalog], ignore_index = True, axis=1)
new_catalog = pd.concat([new_catalog, clusters3], ignore_index = True, axis=1)
new_catalog = new_catalog.drop(columns=[1])
new_catalog = new_catalog.rename({0: "item_id", 2: "clear_name", 3: "tokens", 4: "cluster"}, axis=1)

In [19]:
new_catalog.sort_values(by="cluster")[:55]
# wv.most_similar("кабачrb_NOUN")
# check that last alph is not ыы

,item_id,clear_name,tokens,cluster
0,41728,картофель,[картофель_NOUN],0
28,65478,картофель,[картофель_NOUN],0
24,64336,картофель,[картофель_NOUN],0
77,1713,картофель,[картофель_NOUN],0
53,76736,картофель,[картофель_NOUN],0
93,18941,картофель,[картофель_NOUN],0
37,71133,картофель запекания,"[картофель_NOUN, запекания_NOUN]",0
139,36093,картофель,[картофель_NOUN],0
118,22714,картофель,[картофель_NOUN],0
1,43208,гроздь черри томаты,"[гроздь_NOUN, черри_NOUN, томаты_NOUN]",1
